# 1. Exploratory Data Analysis (EDA)

## 1.1 LOAD DATA

In [43]:
# Libraries
# -- Stdlib & setup
import os, sys, warnings, json, re
from pathlib import Path
from datetime import datetime
warnings.filterwarnings("ignore")
# -- Ciencia de datos base
import numpy as np
import pandas as pd
from scipy import stats as ss
# -- Visualización
import matplotlib.pyplot as plt
import seaborn as sns
#  -- Interactivas (opcional; idk si las usaremos pero jic)
import plotly.express as px
import plotly.graph_objects as go
# --- Utilidades
from IPython.display import display
from tqdm import tqdm
import joblib

In [44]:
raw = pd.read_csv("../data/train_timeseries.csv")
df0 = raw.copy() # copia de seguridad

## 1.2 STRUCTURE

In [45]:
df0.head() # vista general rápida

,Unnamed: 0,date,unique_id,city_id,shop_id,item_category_id,item_id,monthly_average_price,monthly_sales
0,0,2013-01-31,12_11365,16,12,9,11365,1511.512626,30.0
1,1,2013-01-31,12_11369,16,12,9,11369,572.715278,30.0
2,2,2013-01-31,12_11370,16,12,9,11370,699.776786,30.0
3,3,2013-01-31,12_11373,16,12,9,11373,548.900000,6.0
4,4,2013-01-31,12_12231,16,12,49,12231,350.000000,3.0


In [46]:
df0.shape # dimensión

(14484, 9)

In [47]:
list(df0.columns)

['Unnamed: 0',
 'date',
 'unique_id',
 'city_id',
 'shop_id',
 'item_category_id',
 'item_id',
 'monthly_average_price',
 'monthly_sales']

In [48]:
df0.info # info tipos y nulos

<bound method DataFrame.info of        Unnamed: 0        date unique_id  city_id  shop_id  item_category_id  \
0               0  2013-01-31  12_11365       16       12                 9   
1               1  2013-01-31  12_11369       16       12                 9   
2               2  2013-01-31  12_11370       16       12                 9   
3               3  2013-01-31  12_11373       16       12                 9   
4               4  2013-01-31  12_12231       16       12                49   
...           ...         ...       ...      ...      ...               ...   
14479       14479  2015-10-31    7_5822       21        7                35   
14480       14480  2015-10-31    7_6185       21        7                30   
14481       14481  2015-10-31    7_6497       21        7                28   
14482       14482  2015-10-31    7_7856       21        7                28   
14483       14483  2015-10-31    7_7893       21        7                 6   

       item_id  mon

In [49]:
df0.dtypes.value_counts() # info columnas por tipo

int64      5
object     2
float64    2
Name: count, dtype: int64

In [50]:
df0.columns.str.contains('Unnamed') # columnas que contienen 'Unnamed'

array([ True, False, False, False, False, False, False, False, False])

In [51]:
df0.describe().T # resumen estadístico columnas numéricas

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,14484.0,7241.500000,4181.314985,0.000000,3620.75,7241.5,10862.25,14483.000000
city_id,14484.0,10.833333,6.227364,0.000000,7.00,7.0,16.00,23.000000
shop_id,14484.0,31.967136,15.236351,3.000000,22.00,31.0,44.00,59.000000
item_category_id,14484.0,39.356808,20.439654,2.000000,30.00,35.0,55.00,83.000000
item_id,14484.0,8626.431925,5905.273346,32.000000,4248.00,6457.0,12551.00,22088.000000
monthly_average_price,13262.0,916.089314,820.945949,4.928571,299.00,649.0,1149.00,9305.121212
monthly_sales,13756.0,7.337889,7.915032,0.000000,2.00,5.0,9.00,30.000000


In [54]:
df0.dtypes

Unnamed: 0                        int64
date                     datetime64[ns]
unique_id                        object
city_id                           int64
shop_id                           int64
item_category_id                  int64
item_id                           int64
monthly_average_price           float64
monthly_sales                   float64
dtype: object